In [1]:
from sympy import *
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
#general purpose code for interior cases

phi1 =Symbol('\\phi_1') #phi_2
phi2 =Symbol('\\phi_2') #phi_2
p1=Symbol('p_1') #p1 #TODO: integrate this into the formula*****

cost =Symbol('\\lambda') #can't name variable lambda
out = Symbol('\\pi') #outside option: pi_h(g\i)
beta1  = .5
beta2  = .5
mc1 =Symbol('c_1') 
mc2=Symbol('c_2')

In [3]:
def calc_price(phi1, phi2, cost ,mc1,mc2):
    """compute each insurer's price (assumes interior solution)"""
    p1 = (2*(phi1+mc1)+phi2+mc2)/3 + cost
    p2 = (2*(phi2+mc2)+phi1+mc1)/3 + cost
    return p1, p2
  
    
def calc_s(phi1, phi2, cost,  mc1,mc2):
    """compute the proportion of people choosing each insurer
    assumes interior solution"""
    p1, p2 = calc_price(phi1, phi2, cost, mc1,mc2)
    t_hat = S(1)/2 + (p2 - p1)/(2*cost)
    return t_hat


def calc_profits(phi1,phi2,cost,  mc1,mc2):
    s = calc_s(phi1, phi2, cost,  mc1,mc2)
    p1,p2 = calc_price(phi1, phi2, cost, mc1,mc2)

    profits1, profits2 = s*(p1-phi1-mc1), (1-s)*(p2-phi2-mc2)
    hosp_profit = s*(phi1) +  (1-s)*(phi2)

    return hosp_profit, profits1, profits2

#arbitrary outside option...
def nash_in_nash2(phi1, phi2, cost, mc1,mc2,beta1,out):
    hosp_profit, profits1, profits2 = calc_profits(phi1, phi2,  cost,  mc1,mc2)
    return (hosp_profit-out)*(profits1)



In [4]:
def solve_phi(phi1, phi2, cost, mc1,mc2,beta2,out):
    foc =  diff(nash_in_nash2(phi2, phi1, cost, mc2,mc1,beta2,out), phi2)
    foc = foc.subs( {out:(phi1*calc_s(phi1, phi2, cost,  mc1,mc2))} )
    h =solve(foc,phi2)
    print(len(h))
    print('foc',simplify(foc))
    print('h',h[0])
    
    foc_firm1 =  diff(nash_in_nash2(phi1, phi2, cost, mc1,mc2,beta1,out), phi1)
    foc_firm1 = foc_firm1.subs({out: (phi2*calc_s(phi2, phi1, cost,  mc2, mc1)) })
    foc_firm1 = foc_firm1.subs({phi2:h[0]})

    result  = solve(foc_firm1,phi1)
    return result


result1 = solve_phi(phi1, phi2, cost, mc1,mc2,beta1,out)
print(result1)
result2 = solve_phi(phi2, phi1, cost, mc2,mc1,beta2,out)
print(result2)

2
foc (3*\lambda + \phi_1 - \phi_2 + c_1 - c_2)**2*(3*\lambda + 2*\phi_1 - 4*\phi_2 + c_1 - c_2)/(108*\lambda**2)
h 3*\lambda/4 + \phi_1/2 + c_1/4 - c_2/4
[3*\lambda/2 - c_1/6 + c_2/6, 15*\lambda/2 - 3*c_1/2 + 3*c_2/2]
2
foc (3*\lambda - 4*\phi_1 + 2*\phi_2 - c_1 + c_2)*(3*\lambda - \phi_1 + \phi_2 - c_1 + c_2)**2/(108*\lambda**2)
h 3*\lambda/4 + \phi_2/2 - c_1/4 + c_2/4
[3*\lambda/2 + c_1/6 - c_2/6, 15*\lambda/2 + 3*c_1/2 - 3*c_2/2]


In [7]:
print(float(result1[0].subs({mc1:1,mc2:0,cost:5})))
print(float(result2[0].subs({mc1:1,mc2:0,cost:5})))
print(simult_bargain(25,25, 5,25,[1,0], active=False,betas=[.5,.5]))

print(float(result1[0].subs({mc1:0,mc2:1,cost:5})))
print(float(result2[0].subs({mc1:0,mc2:1,cost:5})))
print(simult_bargain(25,25, 5,25,[0,1], active=False,betas=[.5,.5]))


7.333333333333333
7.666666666666667
(array([7.33344009]), array([7.6667711]))
7.666666666666667
7.333333333333333
(array([7.66676363]), array([7.33335719]))


In [6]:
passive_result = pd.read_csv('../GMM_est/fake_data/passive_data_3.csv',index_col=0)
print(passive_result['phi1'].mean())

passive_result = pd.read_csv('../GMM_est/fake_data/passive_data_4.csv',index_col=0)
print(passive_result['phi1'].mean())

12.76886315841047
18.390389989579386
